In [2]:
import zipfile
zip_path = '/content/drive/MyDrive/facecroppedexpression.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall('/content/expression')

In [5]:
import os
import cv2
import numpy as np
import torch
import shutil
from tqdm import tqdm

# Define paths (modify these based on your folder structure)
input_folder = '/content/expression/angry'  # Input folder with images
output_folder = '/content/expression/happy1'  # Output folder for happy faces

# Create output directory if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Check if CUDA is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load pre-trained face detector model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load pre-trained emotion detection model
# We'll use the facial landmark detector to identify smiles
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

def has_happy_face(image_path):
    """Check if there are happy faces in the image"""

    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not read image {image_path}")
        return False

    # Convert to grayscale for detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Check each face for a smile
    for (x, y, w, h) in faces:
        # Extract face region
        face_roi = gray[y:y+h, x:x+w]

        # Detect smiles within the face region
        smiles = smile_cascade.detectMultiScale(
            face_roi,
            scaleFactor=1.5,
            minNeighbors=15,
            minSize=(25, 25)
        )

        # If smile detected, consider it a happy face
        if len(smiles) > 0:
            return True

    return False

def process_images():
    """Process all images in the input folder and move those with happy faces"""

    if not os.path.exists(input_folder):
        print(f"Error: Input folder {input_folder} does not exist")
        return

    # Get all image files
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = [f for f in os.listdir(input_folder)
                  if any(f.lower().endswith(ext) for ext in image_extensions)]

    print(f"Found {len(image_files)} images to process")

    moved_images = 0

    # Process each image
    for img_file in tqdm(image_files):
        img_path = os.path.join(input_folder, img_file)

        # Check if image contains happy face
        if has_happy_face(img_path):
            # Move the file to output folder
            dest_path = os.path.join(output_folder, img_file)
            shutil.move(img_path, dest_path)
            moved_images += 1

    print(f"Processed {len(image_files)} images")
    print(f"Moved {moved_images} images with happy faces to {output_folder}")

if __name__ == "__main__":
    # Run the face detection process
    process_images()
    print("Processing complete!")

Using device: cuda
Found 8605 images to process


100%|██████████| 8605/8605 [20:16<00:00,  7.08it/s]

Processed 8605 images
Moved 2681 images with happy faces to /content/expression/happy1
Processing complete!


In [6]:
import shutil
import os

source_folders = ['/content/expression/frustration', '/content/expression/exhausted']
destination_folder = '/content/expression/angry'

for source_folder in source_folders:
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)
        shutil.move(source_path, destination_path)

print("Files moved successfully!")

Files moved successfully!


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms, models
from PIL import Image
import os
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset class
class ExpressionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Modified to include 3 classes: angry, happy, sad
        self.classes = ['angry', 'happy', 'sad']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = []
        self.labels = []

        for cls in self.classes:
            class_dir = os.path.join(root_dir, cls)
            for img_name in os.listdir(class_dir):
                if img_name.endswith(('.jpg', '.png', '.jpeg')):
                    self.images.append(os.path.join(class_dir, img_name))
                    self.labels.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Enhanced data transforms with more augmentation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load full dataset
full_dataset = ExpressionDataset(root_dir='/content/expression/', transform=None)

# Split dataset into train, validation, and test sets with better stratification
indices = list(range(len(full_dataset)))
train_idx, temp_idx = train_test_split(indices, test_size=0.3, stratify=full_dataset.labels, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=[full_dataset.labels[i] for i in temp_idx], random_state=42)

# Create subsets with appropriate transforms
train_dataset = Subset(ExpressionDataset(root_dir='/content/expression/', transform=train_transforms), train_idx)
val_dataset = Subset(ExpressionDataset(root_dir='/content/expression/', transform=val_test_transforms), val_idx)
test_dataset = Subset(ExpressionDataset(root_dir='/content/expression/', transform=val_test_transforms), test_idx)

# Create data loaders with appropriate batch sizes
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

# Define improved model architecture
class ExpressionRecognitionModel(nn.Module):
    def __init__(self, num_classes=3):  # Updated to 3 classes
        super(ExpressionRecognitionModel, self).__init__()
        # Load pre-trained ResNet50 instead of VGG16
        self.backbone = models.resnet50(weights='IMAGENET1K_V2')

        # Freeze early layers
        for name, param in self.backbone.named_parameters():
            if 'layer4' not in name and 'fc' not in name:
                param.requires_grad = False

        # Replace the final fully connected layer
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        # Custom classifier with dropout and batch normalization
        self.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )

        # Attention mechanism for focusing on important facial features
        self.attention = nn.Sequential(
            nn.Conv2d(2048, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 1, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Extract features from the backbone
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)  # [batch_size, 2048, 7, 7]

        # Apply attention mechanism
        attention_weights = self.attention(x)
        x = x * attention_weights

        # Global average pooling
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)

        # Apply classifier
        x = self.classifier(x)
        return x

# Initialize model
model = ExpressionRecognitionModel(num_classes=3).to(device)  # Updated to 3 classes

# Define loss function with label smoothing for better generalization
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Optimizer with decoupled weight decay
optimizer = optim.AdamW([
    {'params': [p for n, p in model.named_parameters() if 'backbone' in n], 'lr': 1e-5},
    {'params': [p for n, p in model.named_parameters() if 'backbone' not in n], 'lr': 1e-4}
], weight_decay=1e-2)

# Learning rate scheduler with warmup
def get_lr_scheduler(optimizer):
    lr_scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=[1e-4, 5e-4],
        steps_per_epoch=len(train_loader),
        epochs=40,
        pct_start=0.2,  # Warm up for 20% of training
        anneal_strategy='cos',
        div_factor=25.0,
        final_div_factor=1000.0
    )
    return lr_scheduler

scheduler = get_lr_scheduler(optimizer)

# Mixed precision training with compatible syntax
scaler = torch.cuda.amp.GradScaler()

# Training and validation functions
def train_one_epoch(model, train_loader, criterion, optimizer, scheduler, device, scaler):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()

        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        # Update learning rate at each step
        scheduler.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels

# Implementation of mixup augmentation
def mixup_data(x, y, alpha=0.2):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# Training loop - run for all 40 epochs (removed early stopping)
num_epochs = 40
best_val_acc = 0.0
train_losses, val_losses, test_losses = [], [], []
train_accs, val_accs, test_accs = [], [], []

for epoch in range(num_epochs):
    # Apply mixup training for every other epoch
    use_mixup = (epoch % 2 == 0)

    # Training with or without mixup
    if use_mixup:
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Apply mixup
            inputs, targets_a, targets_b, lam = mixup_data(inputs, labels)

            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            # Update learning rate
            scheduler.step()

            running_loss += loss.item()

            # For accuracy calculation with mixup
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (lam * (predicted == targets_a).sum().item() +
                        (1 - lam) * (predicted == targets_b).sum().item())

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
    else:
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion,
                                                optimizer, scheduler, device, scaler)

    # Validation
    val_loss, val_acc, val_preds, val_labels = validate(model, val_loader, criterion, device)

    # Test - added test evaluation for each epoch
    test_loss, test_acc, test_preds, test_labels = validate(model, test_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    test_losses.append(test_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    test_accs.append(test_acc)

    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')  # Added test accuracy display

    # Save best model but don't stop early
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), '/content/best_model.pth')

# Load best model and evaluate on test set
model.load_state_dict(torch.load('/content/best_model.pth'))
test_loss, test_acc, test_preds, test_labels = validate(model, test_loader, criterion, device)
print(f'Best Model Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

# Plot training metrics
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.plot(test_losses, label='Test Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Val Accuracy')
plt.plot(test_accs, label='Test Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(test_accs, label='Test Accuracy')
plt.title('Test Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.savefig('/content/training_metrics.png')

# Plot confusion matrix for test set
cm = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Angry', 'Happy', 'Sad'],
            yticklabels=['Angry', 'Happy', 'Sad'])
plt.title('Confusion Matrix (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('/content/confusion_matrix_test.png')

print(f'Best Validation Accuracy: {best_val_acc:.2f}%')
print(f'Final Test Accuracy: {test_acc:.2f}%')

# Evaluate class-wise performance
class_accuracy = {}
for i, class_name in enumerate(['Angry', 'Happy', 'Sad']):
    class_indices = [j for j, label in enumerate(test_labels) if label == i]
    if class_indices:
        class_preds = [test_preds[j] for j in class_indices]
        class_true = [test_labels[j] for j in class_indices]
        acc = accuracy_score(class_true, class_preds) * 100
        class_accuracy[class_name] = acc
        print(f"{class_name} Class Accuracy: {acc:.2f}%")

Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 94.2MB/s]
<ipython-input-8-020e63628b15>:174: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-8-020e63628b15>:279: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/40
Train Loss: 0.9961, Train Acc: 55.72%
Val Loss: 0.8978, Val Acc: 62.37%
Test Loss: 0.9141, Test Acc: 61.83%


<ipython-input-8-020e63628b15>:187: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 2/40
Train Loss: 0.9260, Train Acc: 61.60%
Val Loss: 0.8707, Val Acc: 64.39%
Test Loss: 0.8845, Test Acc: 63.39%
Epoch 3/40
Train Loss: 0.9290, Train Acc: 61.60%
Val Loss: 0.8523, Val Acc: 65.89%
Test Loss: 0.8667, Test Acc: 66.27%
Epoch 4/40
Train Loss: 0.8891, Train Acc: 64.68%
Val Loss: 0.8532, Val Acc: 66.06%
Test Loss: 0.8703, Test Acc: 66.16%
Epoch 5/40
Train Loss: 0.9022, Train Acc: 63.55%
Val Loss: 0.8470, Val Acc: 66.43%
Test Loss: 0.8606, Test Acc: 66.19%
Epoch 6/40
Train Loss: 0.8622, Train Acc: 66.15%
Val Loss: 0.8181, Val Acc: 68.21%
Test Loss: 0.8363, Test Acc: 67.32%
Epoch 7/40
Train Loss: 0.8750, Train Acc: 65.28%
Val Loss: 0.8234, Val Acc: 67.62%
Test Loss: 0.8342, Test Acc: 67.27%
Epoch 8/40
Train Loss: 0.8462, Train Acc: 67.27%
Val Loss: 0.8110, Val Acc: 68.88%
Test Loss: 0.8316, Test Acc: 68.08%
Epoch 9/40
Train Loss: 0.8747, Train Acc: 65.33%
Val Loss: 0.8256, Val Acc: 68.62%
Test Loss: 0.8405, Test Acc: 68.48%
Epoch 10/40
Train Loss: 0.8294, Train Acc: 67.92